In [1]:
import numpy as np
from lcode2dPy import DiagnosticsFXi, DiagnosticsColormaps, DiagnosticsTransverse, SaveRunState, Cartesian3dSimulation
from config import config

In [14]:
# We create a separate config that will allow us to create a plasma state with
# only one ion and an electron.
config_create_plasmastate = config.copy()
config_create_plasmastate['processing-unit-type'] = 'cpu'
config_create_plasmastate['window-length'] = -config_create_plasmastate['xi-step']
grid_step_size = config_create_plasmastate['window-width-step-size']

In [ ]:
# First, we create a plasmastate with all the required arrays:
diag = [SaveRunState(output_period=0, save_plasma=True,
                     unique_directory_name='_initial')]
beam_parameters = {'beam':{'ampl':0}}
sim = Cartesian3dSimulation(config=config_create_plasmastate, diagnostics=diag,
                            beam_parameters=beam_parameters)

sim.step()

plasmastate_initial = np.load('run_states_initial/plasmastate_00000.00.npz')

In [16]:
from lcode2dPy.plasma3d.initialization import get_deposit_plasma
from lcode2dPy.config.config import Config
from lcode2dPy.plasma3d.data import Arrays

initial_deposition = get_deposit_plasma(Config(config_create_plasmastate))

In [17]:
# Now we create a single ion macroparticle:
# Here you can set the position, charge and mass of the ion macroparticle:
x_init_ion, y_init_ion = 1., 1.
q_ion, m_ion = 1. * grid_step_size ** 2, 1. * grid_step_size ** 2

ion_particle = Arrays(xp=np, x_init=x_init_ion, y_init=y_init_ion,
                      q=q_ion, m=m_ion,
                      x_offt=0., y_offt=0., px=0., py=0., pz=0.)

# Here we create the charge density distribution of the single ion macroparticle:
rho_of_one_ion = initial_deposition(ion_particle, Arrays(xp=np))[0]

In [18]:
# import matplotlib.pyplot as plt

# plt.imshow(rho_of_one_ion)
# plt.colorbar()

In [9]:
# Now we create a single electron:
x_init, y_init = x_init_ion, y_init_ion
q, m = -.1 * grid_step_size ** 2, 1. * grid_step_size ** 2
px, py, pz = 0.1, 0., 0.

particles = Arrays(np, x_init=x_init, y_init=y_init,
                   x_offt=0., y_offt=0., px=px, py=py, pz=pz,
                   q=q, m=m,
                   dx_chaotic=0., dy_chaotic=0.,
                   dx_chaotic_perp=0., dy_chaotic_perp=0.)

In [19]:
# If we don't need electrons:
# particles = Arrays(np, x_init=0., y_init=0.,
#                    x_offt=0., y_offt=0., px=0., py=0., pz=0.,
#                    q=0., m=0.,
#                    dx_chaotic=0., dy_chaotic=0.,
#                    dx_chaotic_perp=0., dy_chaotic_perp=0.)

In [20]:
np.savez_compressed(
    file='plasmastate_with_one_ion',
    ro_initial=rho_of_one_ion,

    xi_plasma_layer=np.array([-config_create_plasmastate['xi-step']]),
    x_init=particles.x_init, y_init=particles.y_init,
    x_offt=particles.x_offt, y_offt=particles.y_offt,
    px=particles.px, py=particles.py, pz=particles.pz,
    q=particles.q, m=particles.m,
    
    dx_chaotic=particles.dx_chaotic,
    dy_chaotic=particles.dy_chaotic,
    dx_chaotic_perp=particles.dx_chaotic_perp,
    dy_chaotic_perp=particles.dy_chaotic_perp,

    Ex=plasmastate_initial['Ex'], Ey=plasmastate_initial['Ey'], Ez=plasmastate_initial['Ez'],
    Bx=plasmastate_initial['Bx'], By=plasmastate_initial['By'], Bz=plasmastate_initial['Bz'],

    Phi=plasmastate_initial['Phi'], ro=plasmastate_initial['ro'],
    jx=plasmastate_initial['jx'], jy=plasmastate_initial['jy'], jz=plasmastate_initial['jz']
)

In [ ]:
diag = [DiagnosticsTransverse(output_period=0, saving_xi_period=1,
                              colormaps='rho', colormaps_type='pictures')]
beam_parameters = {'beam':{'ampl':0}}
sim = Cartesian3dSimulation(config=config, diagnostics=diag,
                            beam_parameters=beam_parameters)

sim.external_plasmastate = True
sim.path_to_plasmastate = 'plasmastate_with_one_ion.npz'

sim.step()